In [ ]:
pip install transformers


In [ ]:
pip install emoji

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import sklearn
import pandas as pd
from transformers import LineByLineTextDataset

In [ ]:
train = open('test_text.txt',"r")
dataset = []
for i in train:
    dataset.append(i.splitlines())
  

In [ ]:
dataset[0]

['@user Can U Help?||More conservatives needed on #TSU + get paid 4 posting stuff like this!||YOU $ can go to ']

In [ ]:
len(dataset)

784

In [ ]:
task='irony'
MODEL = f"cardiffnlp/bertweet-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [ ]:
!pip install tweet-preprocessor

In [ ]:
import re
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")



In [ ]:
import preprocessor as p


In [ ]:
def preprocess(text):
    new_text = []
    for t in text:
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        t = p.clean(t)
        t = REPLACE_NO_SPACE.sub("", t.lower())
        t = REPLACE_WITH_SPACE.sub(" ", t)
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)


In [ ]:
preprocessed_data = []
for i in dataset:
  preprocessed_data.append(preprocess(i))

In [ ]:
preprocessed_data[5]

'corny jokes are my absolute favorite'

In [ ]:
scores = []
for i in preprocessed_data:
  encoded_input = tokenizer(i,return_tensors='pt')
  output = model(**encoded_input)
  score = output[0][0].detach().numpy()
  score = softmax(score)
  scores.append(score)

In [ ]:
dataset_val_l = open('test_labels.txt','r', encoding='utf-8')
dataset_l = []
for i in dataset_val_l.readlines():
    dataset_l.append(int(i))
dataset_val_l.close()

In [ ]:
p_labels = []
for i in scores:
  p_labels.append(np.argmax(i))

In [ ]:
p_labels

[1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,


In [ ]:
from sklearn.metrics import f1_score
f1_score(dataset_l, p_labels)

0.6121919584954604